In [66]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from hog_extractor import HOGExtractor
import random as rand
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
from helpers import print_executed
from tqdm import tqdm
from matplotlib import pyplot as plt
import pandas as pd
import datetime

hog_parameters = {
    'color_model': 'hsv',  # hls, hsv, yuv, ycrcb
    'svc_input_size': 64,  #
    'number_of_orientations': 11,  # 6 - 12
    'pixels_per_cell': 16,  # 8, 16
    'cells_per_block': 2,  # 1, 2
    'do_transform_sqrt': True,
}

start_frame = cv2.imread("./dataset/positive/00001/00000_00000.ppm")
extractor = HOGExtractor(hog_parameters, start_frame)
positive_images, negative_images = [], []
    
print_executed()



 EXECUTED BLOCK AT 19:38:25


In [68]:
# read file which describe positive dataset
positive_data_descr = pd.read_csv('dataset/positive/folders_classes.csv', delimiter=';', dtype=object)
positive_data_descr = positive_data_descr.drop('opis', axis=1)

# select only prohibitory signs which are speed limits
positive_data_descr = \
    positive_data_descr.loc[(positive_data_descr['typ'] == 'b') & (positive_data_descr['numer'] == '33')]
print(positive_data_descr, '\n')

# read file which describe types of signs and select its language
sign_description = pd.read_csv('dataset/sign_types.csv', delimiter=';')
sign_description = sign_description.drop('opis', axis=1)
print(sign_description.head(5), '\n')

  folder typ numer
0  00000   b    33
1  00001   b    33
2  00002   b    33
3  00003   b    33
4  00004   b    33
5  00005   b    33
7  00007   b    33
8  00008   b    33 

  typ    description
0   a        warning
1   b    prohibitory
2   c      mandatory
3   d  informational 



In [70]:
print("Loading negative images to memory...")

negative_paths = glob.glob('./dataset/negative/*/*.png')
for path in tqdm(negative_paths):
    image = cv2.imread(path)
    if image.shape[0] >= 30 or image.shape[1] >= 30:
        negative_images.append(image)

# good_width, good_height, _ = negative_images[0].shape # zastanowić się czy to jest potrzebne!!!

print_executed()


  0%|          | 0/9866 [00:00<?, ?it/s]

  1%|          | 93/9866 [00:00<00:10, 926.26it/s]

Loading negative images to memory...


  2%|▏         | 206/9866 [00:00<00:09, 977.90it/s]

  3%|▎         | 321/9866 [00:00<00:09, 1022.61it/s]

...

 97%|█████████▋| 9601/9866 [00:09<00:00, 911.11it/s]

 98%|█████████▊| 9693/9866 [00:09<00:00, 893.33it/s]

 99%|█████████▉| 9784/9866 [00:09<00:00, 896.33it/s]

100%|██████████| 9866/9866 [00:09<00:00, 1041.93it/s]


 EXECUTED BLOCK AT 19:39:22


In [71]:
print("Loading positive images to memory...\n")

# for each selected folder
for folder in positive_data_descr['folder']:
    # read all images from it and sort it
    paths = sorted(glob.glob('./dataset/positive/' + folder + '/*.ppm'))
    # read csv which contains bounding boxes of traffic sign
    csv = pd.read_csv('./dataset/positive/' + folder + '/GT-' + folder + '.csv', delimiter=';')
    print("\nReading folder: ", folder, '\n')
    for path in tqdm(paths):
        # read name of the image
        name = path.split('/')[4]
        # search for selected image in the csv and flatten that row to list
        row = (csv.loc[csv['Filename'] == name]).values.tolist()
        # read image and cut ROI with traffic sign, then add to list
        image = cv2.imread(path)
        if image.shape[0] >= 35 or image.shape[1] >= 35:
            image = image[row[0][3]:row[0][5], row[0][4]:row[0][6]]
            #print(image.shape)
            positive_images.append(image)

print_executed()


  0%|          | 0/210 [00:00<?, ?it/s]

 29%|██▊       | 60/210 [00:00<00:00, 594.02it/s]

Loading positive images to memory...


Reading folder:  00000 



 58%|█████▊    | 121/210 [00:00<00:00, 598.34it/s]

 83%|████████▎ | 175/210 [00:00<00:00, 575.26it/s]

...

 88%|████████▊ | 1244/1410 [00:02<00:00, 590.45it/s]

 92%|█████████▏| 1304/1410 [00:02<00:00, 584.97it/s]

 97%|█████████▋| 1365/1410 [00:02<00:00, 590.22it/s]

100%|██████████| 1410/1410 [00:02<00:00, 532.80it/s]


 EXECUTED BLOCK AT 19:39:51


In [72]:
# stack already read images to np array
# shapes of arrays can be different, because we have stack images with different shapes to one array
positive_images, negative_images = np.asarray(positive_images), np.asarray(negative_images)
total_signs, total_nonsigns = positive_images.shape[0], negative_images.shape[0]
1
print("Positive images shape: ", len(positive_images1), "type: ", type(positive_images1))
print("Negative images shape: ", len(negative_images1), "type: ", type(negative_images1))
print("Total positive signs: ", total_signs, "\nTotal nonsigns: ", total_nonsigns)

positive_features, negative_features = [], []


Positive images shape:  9183 type:  <class 'numpy.ndarray'>
Negative images shape:  9866 type:  <class 'numpy.ndarray'>
Total positive signs:  9183 
Total nonsigns:  19732


In [73]:
print("Extracting features from traffic signs...")

i = 0
for img in tqdm(positive_images[:100]):
    positive_features.append(extractor.features(img)) 
    
print_executed()


  0%|          | 0/100 [00:00<?, ?it/s]

  8%|▊         | 8/100 [00:00<00:01, 72.00it/s]

Extracting features from traffic signs...


 16%|█▌        | 16/100 [00:00<00:01, 73.26it/s]

 24%|██▍       | 24/100 [00:00<00:01, 73.73it/s]

...

 74%|███████▍  | 74/100 [00:01<00:00, 54.00it/s]

 83%|████████▎ | 83/100 [00:01<00:00, 61.22it/s]

 93%|█████████▎| 93/100 [00:01<00:00, 67.89it/s]

100%|██████████| 100/100 [00:01<00:00, 68.97it/s]


 EXECUTED BLOCK AT 19:40:13


In [74]:
print("Extracting features from non traffic signs...")
# print(len(positive_features), type(positive_features[0]))
i = 0
for img in tqdm(negative_images[:100]):
    negative_features.append(extractor.features(img)) 

print_executed()


  0%|          | 0/100 [00:00<?, ?it/s]

  8%|▊         | 8/100 [00:00<00:01, 72.10it/s]

Extracting features from non traffic signs...


 17%|█▋        | 17/100 [00:00<00:01, 74.88it/s]

 25%|██▌       | 25/100 [00:00<00:00, 75.86it/s]

...

 72%|███████▏  | 72/100 [00:01<00:00, 67.69it/s]

 82%|████████▏ | 82/100 [00:01<00:00, 73.21it/s]

 91%|█████████ | 91/100 [00:01<00:00, 75.50it/s]

100%|██████████| 100/100 [00:01<00:00, 78.25it/s]


 EXECUTED BLOCK AT 19:40:17


In [75]:
positive_features = np.asarray(positive_features)
negative_features = np.asarray(negative_features)

print_executed()


 EXECUTED BLOCK AT 19:40:19


In [76]:
print("Scaling features...")
print(len(positive_features), len(positive_features[0]))
print(len(negative_features), len(negative_features[0]))

unscaled_x = np.vstack((positive_features, negative_features)).astype(np.float64)
scaler = StandardScaler().fit(unscaled_x)
x = scaler.transform(unscaled_x)
# y = np.hstack((np.ones(total_signs), np.zeros(total_nonsigns)))
y = np.hstack((np.ones(100), np.zeros(100)))

print(" x shape: ", x.shape, " y shape: ", y.shape)




Scaling features...
100 1188
100 1188
 x shape:  (200, 1188)  y shape:  (200,)


In [77]:
print("Training classifier and saving models")

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.7, random_state=rand.randint(1, 100))
svc = LinearSVC()
svc.fit(x_train, y_train)
accuracy = svc.score(x_test, y_test)
print("Accuracy: ", np.round(accuracy, 4))
now = datetime.datetime.now()
created = str(now.year)+str(now.month)+str(now.day)  # +str(now.hour)+str(now.minute)

joblib.dump(svc, './trained_models/SVC_'+created+'.pkl')
joblib.dump(scaler, './trained_models/scaler_'+created+'.pkl')


print_executed()


Training classifier and saving models
Accuracy:  1.0

 EXECUTED BLOCK AT 19:40:24


In [78]:
print("Test prediction of single ROI")

f1 = extractor.features(negative_images[rand.randint(0, 2000)])
f2 = extractor.features(positive_images[rand.randint(0, 2000)])
f3 = extractor.features(positive_images[rand.randint(0, 2000)])
f4 = extractor.features(negative_images[rand.randint(0, 2000)])

fA = scaler.transform([f1, f2, f3, f4])
print(svc.predict(fA))


f5 = extractor.features(negative_images[rand.randint(2000, total_nonsigns)])
f6 = extractor.features(positive_images[rand.randint(2000, total_signs)])
f7 = extractor.features(positive_images[rand.randint(2000, total_signs)])
f8 = extractor.features(negative_images[rand.randint(2000, total_nonsigns)])

fB = scaler.transform([f5, f6, f7, f8])
print(svc.predict(fB))



Test prediction of single ROI
[0. 1. 1. 0.]
[0. 1. 1. 0.]
